In [1]:
import  os
import  sys
import  platform
import  time
import  datetime
import  pickle

import  numpy                   as np
import  pandas                  as pd
from    PIL                     import Image, ImageDraw
from    IPython.display         import Image    as IPythonImage

from    argparse                import ArgumentParser
import  matplotlib.pyplot       as plt
import  matplotlib.image        as mpimg

import  tensorflow              as tf
from    tensorflow              import keras
from    tensorflow.keras        import utils
from    tensorflow.keras        import backend  as K
from    tensorflow.keras.layers import Layer

# Pendulum

## Data Reading

In [2]:
inp_zip     = "pend.zip"
inp_path    = "GeneralisedPendulum"         # path and prefix of the input data files
header      = 3                                     # line with header in the input data files

# partitioning of the data to be generated
train_test  = {
    'train' : [ 8, 10, 11, 13, 15, 16, 18 ],
    'test'  : [ 9, 14, 17 ]
}

In [3]:
!wget -O {inp_zip} https://www.dropbox.com/scl/fi/ell5z8bje172n0c6st0e3/pend.zip?rlkey=t5mxsl2cev6xbugogcd3ej028&dl=0

--2024-03-06 10:02:07--  https://www.dropbox.com/scl/fi/ell5z8bje172n0c6st0e3/pend.zip?rlkey=t5mxsl2cev6xbugogcd3ej028
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8cc03045298791e23140764936.dl.dropboxusercontent.com/cd/0/inline/COnM_FG66GTYnr5FKeKtTT32RcEqxr4ibZ75Ksrh77qVDaety_y0AXTnDumVAVJl6Jje_ZWQIKhRQRuS2tKFtVZe_4YaO-kBN600UDrbdzHkuI2iC1obDyOkl2DXwksdTnA/file# [following]
--2024-03-06 10:02:08--  https://uc8cc03045298791e23140764936.dl.dropboxusercontent.com/cd/0/inline/COnM_FG66GTYnr5FKeKtTT32RcEqxr4ibZ75Ksrh77qVDaety_y0AXTnDumVAVJl6Jje_ZWQIKhRQRuS2tKFtVZe_4YaO-kBN600UDrbdzHkuI2iC1obDyOkl2DXwksdTnA/file
Resolving uc8cc03045298791e23140764936.dl.dropboxusercontent.com (uc8cc03045298791e23140764936.dl.dropboxusercontent.com)... 162.125.6.15, 2620:100:6019:15::a27d:40f
Connecting to uc8cc030452

In [4]:
!mkdir {inp_path}
!unzip {inp_zip} -d {inp_path}

Archive:  pend.zip
  inflating: GeneralisedPendulum/GeneralisedPendulum08.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum09.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum10.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum11.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum13.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum14.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum15.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum16.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum17.dta  
  inflating: GeneralisedPendulum/GeneralisedPendulum18.dta  


In [5]:
def read_file_pend( n, points_only=True ):
    """ -------------------------------------------------------------------------------------------------------------
    read the file with the given number in the filename and return a list with the array of coordinates
    of the 3 points in time

    n:              [int] number of the file

    return:         np.array with time as first dimension, and points p1, p2, p3 coordinates
    ------------------------------------------------------------------------------------------------------------- """

    f           = "{}/GeneralisedPendulum{:02d}.dta".format( inp_path, n )
    assert os.path.isfile( f ), "data file {} not found".format( f )
    r           = pd.read_csv( f, sep='\t', header=header )

    if points_only:
        points      = np.array( [
                r[ 'x1' ],  r[ 'y1' ],
                r[ 'x2' ],  r[ 'y2' ],
                r[ 'x3' ],  r[ 'y3' ]
        ] ).T

    else:
        points      = np.array( [
                r[ 'x1' ],  r[ 'y1' ],
                r[ 'x2' ],  r[ 'y2' ],
                r[ 'x3' ],  r[ 'y3' ],
                r[ 'A1x' ], r[ 'A1y' ],
                r[ 'B1x' ], r[ 'B1y' ],
                r[ 'A2x' ], r[ 'A2y' ],
                r[ 'B2x' ], r[ 'B2y' ]
        ] ).T

    return points

In [6]:
def gen_sequences( files, seq_len=20, down=1, batch_size=32, train=True ):
    """ -------------------------------------------------------------------------------------------------------------
    construct a data generator that takes seq_len elements as input, and the following lement as target
    concatenate datasets over all requested files

    files:          [list] of numbers of the data files
    seq_len:        [int] length of the sequence of images
    down:           [int] downsamplig factor, when down=1 all possible sequences will be generated, half with down=2...

    return:         [tf.data.Dataset] that yelds a tuple with batch of inputs and batch of targets
    ------------------------------------------------------------------------------------------------------------- """
    if train:
        tr_ts   = 'train'
    else:
        tr_ts   = 'test'
    files   = train_test[ tr_ts ]

    dset    = None
    for n in files:
        p       = read_file_pend( n )
        inp     = p[ : -seq_len ]       # discard the last elements in the serie for the inputs
        target  = p[ seq_len: ]         # discard the first elements in the serie for the targets
        d       = utils.timeseries_dataset_from_array(
                inp,
                target,
                sequence_length = seq_len,
                sequence_stride = down,
                batch_size      = batch_size,
                shuffle         = train
        )
        if dset is None:
            dset    = d
        else:
            dset    = dset.concatenate( d )

    return dset

In [7]:
def gen_dset_pend( seq_len=20, down=1, batch_size=32 ):
    """ -------------------------------------------------------------------------------------------------------------
    generated the training and test sets

    seq_len:        [int] length of the sequence of images
    down:           [int] downsamplig factor, when down=1 all possible sequences will be generated, half with down=2...

    return:         [tuple] with training and test datasets
    ------------------------------------------------------------------------------------------------------------- """
    files   = train_test[ "train" ]
    tr_set  = gen_sequences( files, seq_len=seq_len, down=down, batch_size=batch_size, train=True )
    files   = train_test[ "test" ]
    ts_set  = gen_sequences( files, seq_len=seq_len, down=down, batch_size=batch_size, train=False )

    return tr_set, ts_set

## Parameters

In [8]:
FRMT                    = "%y-%m-%d_%H-%M-%S"   # datetime format for folder names


# folders and files inside the main execution folder - NOTE the variables will be updated in init_dirs()
dir_current         	= None
dir_res             	= '../res'              # where all results are stored
dir_test            	= 'test'                # where all results are stored
nn_final            	= "nn_final.h5"         # the name of the trained model weights
nn_graph            	= "nn.png"              # the graph of the model
gif_file                = "output.gif"

BATCH_SIZE              = 16                    # batch size of the generator
N_EPOCHS                = 2                     # number of training epochs
LRATE                   = 0.001                 # learning rate
DROPOUT                 = 0.2                   # dropout probability during training of attention
NUM_HEADS               = 4                     # number of attention heads
SEQ_LEN                 = 40                    # length of the time sequence before the prediction
DOWNSAMPLE              = 1                     # downsamplig factor, with down=1 keep all sequences
DATA_DIM                = 6                     # dimension of a data sample (2D coordinates for 3 points)
EMBED_DIM               = 64                    # embedding dimension
EMBED_MODE              = 'time2vec'            # embedding modality - current options are:
                                                #   "conv1d"    use 1D convolution
                                                #   None        no embedding at all
                                                #   "time2vec"  use Time2Vec

## Time2Vec

In [9]:
class Time2Vec( Layer ):
    """
    this class implements  Kazemi et al,'s Time2Vec layer, to be used as possible embedding
    """

    def __init__( self, **kwargs ):
        """
        initialization of the class
        """
        super().__init__( **kwargs )
        self.inp_dim    = DATA_DIM
        self.dim        = EMBED_DIM


    def build( self, input_shape ):
        """
        define all trainable weights
        NOTE a certain pedantry in assigning names to each component
        it has been necessary to get rid of "ValueError: Unable to create dataset (name already exists)"
        when executing model.save_weights()
        """
        self.wa         = self.add_weight(
            shape           = ( self.inp_dim, self.dim - 1 ),
            initializer     = 'uniform',
            name            = "WeightT2VLinearMatrix",
            trainable       = True
        )
        self.ba         = self.add_weight(
            shape=( 1, self.dim - 1 ),
            initializer     = 'uniform',
            name            = "WeightT2VLinearOffset",
            trainable       = True
        )
        self.wb         = self.add_weight(
            shape=( self.inp_dim, 1 ),
            initializer     = 'uniform',
            name            = "WeightT2VPeriodicMatrix",
            trainable       = True
        )
        self.bb         = self.add_weight(
            shape=( 1, 1 ),
            initializer     = 'uniform',
            name            = "WeightT2VPeriodicOffset",
            trainable       = True
        )


    def call( self, inputs ):
        """
        execute the Time2Vec computation
        NOTE the specifications in the tf.tensordot multiplication, for eliminating the
        original dimensions of the tensors (the 6 points coordinates), obtaining the
        desired embedding dimension
        """
        linear          = tf.tensordot( inputs, self.wb, axes=( (-1), (0) ) )
        linear          += self.bb
        periodic        = tf.tensordot( inputs, self.wa, axes=( (-1), (0) ) )
        periodic        += self.ba
        periodic        = tf.math.sin( periodic )
        t2v             = tf.concat( [ linear, periodic ], -1 )
        return t2v

## TransformerDecoder

In [10]:
class TransDecoder( object ):
    """
    this class implements an essential Transformer decoder
    NOTE there is already in Keras a TransformerDecoder layer, but by building it in pieces it is possible
    to isolate the part that implements attention, in order to exctract and visualize attention results
    """

    def __init__( self ):
        """
        initialization of the class
        """
        self.num_heads  = NUM_HEADS
        self.lenght     = SEQ_LEN
        self.inp_dim    = DATA_DIM
        if TRAIN:
            self.dropout    = DROPOUT
        else:
            self.dropout    = 0.0
        if EMBED_MODE is None:
            self.dim        = self.inp_dim
            self.embedding  = self._noembedding()
        elif EMBED_MODE == "conv1d":
            self.dim        = EMBED_DIM
            self.embedding  = self._conv_embed()
        elif EMBED_MODE == "time2vec":
            self.dim        = EMBED_DIM
            self.embedding  = self._t2v_embed()
        self.loss_func  = keras.losses.MeanSquaredError()
        self.key_dim    = self.dim // self.num_heads
        self.attention  = self._attention()
        self.model      = self.create_model()


    def _noembedding( self ):
        """
        do not use any embedding
        """
        embedding   = lambda x: x
        return embedding


    def _conv_embed( self ):
        """
        used conv1D as embedding, in order to apply the same matrix to all vectors in the sequence
        """
        embedding   = keras.layers.Conv1D( self.dim, kernel_size=1, name="Conv1DEmbed" )
        return embedding


    def _t2v_embed( self ):
        """
        used Time2Vec as embedding, note the necessary transformation of input shape
        """
        return Time2Vec( name="Time2VecEmbed" )


    def _loss_func( self, y_true, y_pred ):
        """
        define the loss function for the case when the output of the model is as embedded vector
        """
        msq     = keras.losses.MeanSquaredError()
        y_true  = tf.expand_dims( y_true, axis=0 )
        return msq( self.embedding( y_true ), y_pred )


    def _attention( self ):
        """
        define the attention part of the model
        """
        att         = keras.layers.MultiHeadAttention(
                num_heads   = self.num_heads,
                key_dim     = self.key_dim,
                dropout     = self.dropout,
                name        = "MHAttention"
        )
        return att


    def create_model( self ):
        """
        create the model
        """

        inputs      = keras.layers.Input( shape=( self.lenght, self.inp_dim ), dtype=tf.float32 )
        x           = self.embedding( inputs )
        x           = self.attention( x, x )
        x           = keras.layers.LayerNormalization( name="NormLayer" )( x )
        # note that the Dense output has shape batch_size X lenght X inp_dim
        x           = keras.layers.Dense( self.inp_dim, activation='sigmoid', name="Dense1" )( x )
        x           = keras.layers.Flatten( name="FlattenDense" )( x )
        outputs     = keras.layers.Dense( self.inp_dim, name="Dense2" )( x )
        model       = keras.Model(inputs=inputs, outputs=outputs)

        return model

## Main functions

In [11]:
def create_dset():
    """ -------------------------------------------------------------------------------------------------------------
    create the datasets for training and validation
    ------------------------------------------------------------------------------------------------------------- """
    # global train_set, test_set
    print( "Now creating the datasets...\n" )
    train_set, test_set = gen_dset_pend(
            seq_len     = SEQ_LEN,
            down        = DOWNSAMPLE,
            batch_size  = BATCH_SIZE
    )
    return train_set, test_set

In [12]:
def train_model( nn, train_set, test_set ):
    """ -------------------------------------------------------------------------------------------------------------
    train the model and save the weigths
    ------------------------------------------------------------------------------------------------------------- """
    # global train_set, test_set

    nn.model.compile(
            optimizer   = keras.optimizers.Adam( learning_rate=LRATE ),
            loss        = nn.loss_func,
            metrics     = [ 'accuracy' ]
    )
    hist        = nn.model.fit(
            x                   = train_set,
            validation_data     = test_set,
            epochs              = N_EPOCHS,
            verbose             = 1
    )

    # save the model and its graph
    nn.model.save_weights( nn_final )
    keras.utils.plot_model( nn.model, to_file=nn_graph, show_shapes=True, show_layer_names=True )

    return nn, hist

In [13]:
def test_model( nn, nfile=8 ):
    """ -------------------------------------------------------------------------------------------------------------
    test the model on one file of sequences, and represent the rsults as images
    ------------------------------------------------------------------------------------------------------------- """
    print( f"Now starting testing on file {nfile}...\n" )

    points_all  = read_file_pend( nfile, points_only=False )
    points      = points_all[ :, :6 ]

    preds       = points.copy()
    slen        = SEQ_LEN
    npts        = len( points )

    # now in a loop a full sequence is presented, and the model predict the next
    # trajectory, then the sequnce is shifted in time, with the last element filled with
    # the prediction
    x           = points[ : slen ]      # prepare the first sequence, all made by original points
    for i in range( npts - slen ):
        y                   = nn.model( x[ np.newaxis, ... ] )
        y                   = y[ 0 ].numpy()
        preds[ i + slen ]   = y         # save the current prediction
        x                   = np.roll( x, -1, axis=0 )
        x[ -1 ]             = y         # the last element of the sequence is the prediction

    # call the graphic function that generate the images
    gen_compare( points_all, preds, nfile, gif_file )
    IPythonImage( open( gif_file, 'rb' ).read() )

    return points, preds

## Image output generation

In [14]:
isize       = ( 256, 256 )                          # image size

img_dir     = "imgs/"                               # output images directory
background  = 20                                    # background gray level
foreground  = 160                                   # object filling gray level
linecolor   = 210                                   # object edge gray level
edgecolor   = 250                                   # object edge gray level
fore_true   = "#00b000"
edge_true   = "#00ff00"
fore_pred   = "#b00080"
edge_pred   = "#ff0000"
border      = 0.15                                  # minimum border of the trajectories, as fraction of 1.0

# numbers of existing and valid files
valid_files = [ 8, 9, 10, 11, 13, 14, 15, 16, 17, 18 ]

In [15]:
def read_file_test( n ):
    """ -------------------------------------------------------------------------------------------------------------
    read the file with the given number in the filename

    n:              [int] number of the file

    return:         [dict] [ with keys "p1", "p2", "p3", "A1", "B1", "A2", "B2" and np.arrays as values ]
    ------------------------------------------------------------------------------------------------------------- """

    f           = "{}{:02d}.dta".format( inp_path, n )
    assert os.path.isfile( f ), f"data file {f} not found"

    r           = pd.read_csv( f, sep='\t', header=header )
    d           = {}
    d[ 'p1' ]   = np.array( [ r[ 'x1' ], r[ 'y1' ] ] ).T
    d[ 'p2' ]   = np.array( [ r[ 'x2' ], r[ 'y2' ] ] ).T
    d[ 'p3' ]   = np.array( [ r[ 'x3' ], r[ 'y3' ] ] ).T
    d[ 'A1' ]   = np.array( [ r[ 'A1x' ], r[ 'A1y' ] ] ).T
    d[ 'B1' ]   = np.array( [ r[ 'B1x' ], r[ 'B1y' ] ] ).T
    d[ 'A2' ]   = np.array( [ r[ 'A2x' ], r[ 'A2y' ] ] ).T
    d[ 'B2' ]   = np.array( [ r[ 'B2x' ], r[ 'B2y' ] ] ).T

    return d

In [16]:
def scale( points, mul, off ):
    """ -------------------------------------------------------------------------------------------------------------
    scale points from real to image coordinates, avoiding the figure to move out of the boundaries

    points:         [dict] of np.array with shape ( N, 2 )
    mul:            [np.array] scaling multiplier
    off:            [np.array] scaling offset

    return:         [dict] of np.array with shape ( N, 2 )

    ------------------------------------------------------------------------------------------------------------- """

    fact    = np.array( isize )
    scaled  = {}

    boff    = np.array( ( border, border ) )        # take some border for the trajectories
    bmul    = np.array( ( 1. - 2 * border, 1. - 2 * border ) )
    for k in points.keys():
        p           = points[ k ]
        p           = ( p - off ) / mul             # scale in range 0..1
        p           = np.array( [ 1., -1. ] ) * p   # invert Y
        p           = np.array( [ 0.,  1. ] ) + p   # invert Y
        p           = bmul * p + boff               # allow for borders
        p           = fact * p                      # scale up to pixels
        scaled[ k ] = p.astype( int )

    return scaled

In [17]:
def scaling( points ):
    """ -------------------------------------------------------------------------------------------------------------
    computing the scaling factors for a given set of points

    points:         [dict] of np.array with shape ( N, 2 )

    return:         [tuple] mul, off

    ------------------------------------------------------------------------------------------------------------- """
    fact    = np.array( isize )
    scaled  = {}

    max_x   = 0.0
    max_y   = 0.0
    min_x   = 1.0
    min_y   = 1.0
    for p in points.keys():
        mx, my      = points[ p ].min( axis=0 )
        min_x       = min( min_x, mx )
        min_y       = min( min_y, my )
        mx, my      = points[ p ].max( axis=0 )
        max_x       = max( max_x, mx )
        max_y       = max( max_y, my )

    mul     = max( max_x - min_x, max_y - min_y )   # note: should NOT scale X and Y differently
    off     = np.array( ( min_x, min_y ) )          # while the offsets can differ for X and Y

    return mul, off

In [18]:
def draw_compare( true, preds, rod1, rod2 ):
    """ -------------------------------------------------------------------------------------------------------------
    draw one frame with true and predicted triangles

    f:              [str] filename
    true:           [list] [ ( p1x, p1y ), ( p2x, p2y ), ( p3x, p3y ) ]
    preds:          [list] [ ( p1x, p1y ), ( p2x, p2y ), ( p3x, p3y ) ]

    ------------------------------------------------------------------------------------------------------------- """

    img     = Image.new( 'RGB', isize, color=background )
    drw     = ImageDraw.Draw( img )
    drw.polygon( true, fill=None, outline=edge_true, width=8 )
    drw.polygon( preds, fill=None, outline=edge_pred, width=6 )
    drw.polygon( rod1, fill=None, outline="#ffffff", width=6 )
    drw.polygon( rod2, fill=None, outline="#ffffff", width=6 )

    return img

In [19]:
def gen_compare( points, preds, n, fname ):
    """ -------------------------------------------------------------------------------------------------------------
    generate frames for one file

    n:              [int] number of the file
    points:         [dict] { p1, p2, p3, A1, B1, A2, B2 } as returned by read_file_test()
    ------------------------------------------------------------------------------------------------------------- """

    n_frames        = len( points )                             # extract number of time samples
    p_true          = {}
    p_pred          = {}
    p_rod1          = {}
    p_rod2          = {}
    p_true[ "p1" ]  = points[ :, 0:2 ]
    p_true[ "p2" ]  = points[ :, 2:4 ]
    p_true[ "p3" ]  = points[ :, 4:6 ]
    p_pred[ "p1" ]  = preds[ :, 0:2 ]
    p_pred[ "p2" ]  = preds[ :, 2:4 ]
    p_pred[ "p3" ]  = preds[ :, 4:6 ]
    p_true[ "A1" ]  = points[ :, 6:8 ]
    p_true[ "B1" ]  = points[ :, 8:10 ]
    p_true[ "A2" ]  = points[ :, 10:12 ]
    p_true[ "B2" ]  = points[ :, 12:14 ]

    mul, off        = scaling( p_true )                         # compute the proper scaling
    sp_true         = scale( p_true, mul, off )                 # scaled points
    sp_pred         = scale( p_pred, mul, off )                 # scaled points

    img_list        = []

    for i in range( n_frames ):
        t123    = [ sp_true[ 'p1' ][ i ], sp_true[ 'p2' ][ i ], sp_true[ 'p3' ][ i ] ]
        p123    = [ sp_pred[ 'p1' ][ i ], sp_pred[ 'p2' ][ i ], sp_pred[ 'p3' ][ i ] ]
        r1      = [ sp_true[ 'A1' ][ i ], sp_true[ 'B1' ][ i ] ]
        r2      = [ sp_true[ 'A2' ][ i ], sp_true[ 'B2' ][ i ] ]
        xyt     = [ tuple( p ) for p in t123 ]
        xyp     = [ tuple( p ) for p in p123 ]
        xy1     = [ tuple( p ) for p in r1 ]
        xy2     = [ tuple( p ) for p in r2 ]

        img     = draw_compare( xyt, xyp, xy1, xy2 )
        img_list.append( img )

    img_list[ 0 ].save( fname, save_all=True, append_images=img_list[ 1: ], duration=100, loop=0 )

## Usage

In [20]:
nn_t2v  = "pend_t2v_e2000.h5"
!wget -O {nn_t2v} https://www.dropbox.com/scl/fi/fgsfklo5z944acm6l0nu3/pend_t2v_e2000.h5?rlkey=ej6uqo6tpggmtn204r88vg6fu&dl=0

--2024-03-06 10:02:09--  https://www.dropbox.com/scl/fi/fgsfklo5z944acm6l0nu3/pend_t2v_e2000.h5?rlkey=ej6uqo6tpggmtn204r88vg6fu
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4c60ead54581db9e33b64a1d09.dl.dropboxusercontent.com/cd/0/inline/COlhRsa8JrPrZNN-0FvS6hJvQxSF_FwkdDIA70d9N1vF8ugfEkdXay7rVikqsDkxMjU0kdyVMsLqT8DU6b3BVgdj_aVgga_0YKISoxfZbrgiImqeGk2qHHgZB9eNLCDeCJw/file# [following]
--2024-03-06 10:02:10--  https://uc4c60ead54581db9e33b64a1d09.dl.dropboxusercontent.com/cd/0/inline/COlhRsa8JrPrZNN-0FvS6hJvQxSF_FwkdDIA70d9N1vF8ugfEkdXay7rVikqsDkxMjU0kdyVMsLqT8DU6b3BVgdj_aVgga_0YKISoxfZbrgiImqeGk2qHHgZB9eNLCDeCJw/file
Resolving uc4c60ead54581db9e33b64a1d09.dl.dropboxusercontent.com (uc4c60ead54581db9e33b64a1d09.dl.dropboxusercontent.com)... 162.125.4.15, 2620:100:6019:15::a27d:40f
Connecting to u

In [21]:
dir_test    = "test"
if not os.path.exists( dir_test ):
    os.makedirs( dir_test )

In [22]:
TRAIN   = False

In [23]:
nn      = TransDecoder()

In [24]:
train_set, test_set = create_dset()

Now creating the datasets...



In [25]:
# train
# nn, hist    = train_model( nn, train_set, test_set )

In [26]:
# load model

nn.model.load_weights( nn_t2v )

In [27]:
NSEQ    = 8

In [28]:
points, pred    = test_model( nn, nfile=NSEQ )

Now starting testing on file 8...



In [ ]:
IPythonImage( open( gif_file, 'rb' ).read() )